# **Bag of Words**
---
> **Asignatura:** Minería de Datos
>
> **Alumna:** Claudia Luz Rojas Soto
>
> **Código:** 171805



## **Algoritmos:**
---

In [1]:
# Librerias
import re
import math

### 1. BoW
---

In [2]:
def quitarPuntuacion(texto):
    """Permite eliminar puntuación de un texto(incluidas letras con acentos)
    Args:
        texto (string): Texto a preprocesar.
    Returns:
        str: El mismo string pero sin signos de puntuación y tildes.
    """
    return re.sub(r'[^A-Za-z0-9 ]', '', texto).strip().lower() 

In [3]:
def BagOfWords(textos):
    """Permite hallar el BagOfWords de un conjunto de textos(corpus)
    Args:
        textos (List): Lista de textos.
    Returns:
        str: Una lista de palabras de todo el corpus
    """
    bow = textos.flatMap(lambda x: x.split()).filter(lambda y: len(y)>2)
    return bow

In [4]:
textos = ['Hola! hola hola como estas?'
          ,'Hola buenos dias, bien bien y tu como estas?'
          ,'Hola, mi nombre es Claudia, y el tuyo?']

textos = sc.parallelize(textos,3)
preprocesado = textos.map(lambda x: quitarPuntuacion(x))
bow = BagOfWords(preprocesado)
print(bow.collect())

['hola', 'hola', 'hola', 'como', 'estas', 'hola', 'buenos', 'dias', 'bien', 'bien', 'como', 'estas', 'hola', 'nombre', 'claudia', 'tuyo']


**Frecuencia de palabras:**

In [5]:
frecuencia = bow.map(lambda x: (x,1)).reduceByKey(lambda x,y: x + y)
print(frecuencia.collect())

[('hola', 5), ('nombre', 1), ('claudia', 1), ('tuyo', 1), ('estas', 2), ('dias', 1), ('bien', 2), ('como', 2), ('buenos', 1)]


### 2. TF (Term Frequency)
---
### $$tf(t,d) = 1 + log(f_{t,d})$$

In [6]:
import numpy as np
def tf(tokens):
    """Permite hallar el TF(Term Frecuency) de un texto.
    Args:
        tokens (List): Lista de palabras.
    Returns:
        str: Una lista de tuplas compuesto por (palabra, TF).
    """
    # Recuperamos los valores unicos de la lista de ilter
    unicos = list(dict.fromkeys(tokens))
    tf = map(lambda x: (x,1+math.log10(tokens.count(x)/len(tokens))),unicos)
    return list(tf)

In [7]:
# Usaremos el preprocesado anterior como corpus
tokens_x_texto = preprocesado.map(lambda x: x.split())
print(tokens_x_texto.collect())

[['hola', 'hola', 'hola', 'como', 'estas'], ['hola', 'buenos', 'dias', 'bien', 'bien', 'y', 'tu', 'como', 'estas'], ['hola', 'mi', 'nombre', 'es', 'claudia', 'y', 'el', 'tuyo']]


In [8]:
TF = tokens_x_texto.map(lambda x: tf(x))
print(TF.collect()[0])

[('hola', 0.7781512503836436), ('como', 0.30102999566398125), ('estas', 0.30102999566398125)]


### 3. TF-IDF
---

In [9]:
# Modulo para contar palabras
def contarPalabras(palabraRDD):
    """Crea un par RDD con cada palabra y su contador.

    Args:
        palabraRDD (RDD de str): RDD de palabras.

    Returns:
        RDD (str, int): Un RDD que consiste en (word, count) tuplas.
    """
    return (palabraRDD
            .map(lambda x: (x,1))
            .reduceByKey(lambda x,y: x + y)
           )

### **IDF** (Inverse Document Frequency): 
    
###    $$idf(t,D) = log(1 + \frac{N}{n_t})$$

In [10]:
# Usaremos el preprocesado como corpus
print(preprocesado.collect())

['hola hola hola como estas', 'hola buenos dias bien bien y tu como estas', 'hola mi nombre es claudia y el tuyo']


In [11]:
def idf(corpus):
    """Permite hallar el IDF(Inverse Document Frecuency) de las palabras de un corpus.
    Args:
        corpus (RDD list): Lista de textos.
    Returns:
        (RDD list): Una lista de tuplas compuesto por (palabra, IDF).
    """
    # Tamaño del corpus
    tam = len(corpus.collect())
    # Recogemos las palabras unicas por documento dentro de todo el corpus
    unicas = corpus.flatMap(lambda x: set(x.split()))
    # Con esto nos aseguramos de contar la aparicion de una plabra dentro de todo el corpus
    # Solo contando el numero de documentos en los que aparece sin contar sus repeticiones
    # dentro de un documento
    contar = contarPalabras(unicas)
    # APlicamos la formula del idf a las tuplas
    IDF = contar.map(lambda x: (x[0],math.log10(1+(tam/x[1]))))
    return IDF

In [12]:
# Ejecutamos la funcion para nuestro coprus
# Y mostramos las 3 palabras con mayos idf
IDF_corpus = idf(preprocesado)
print(IDF_corpus.takeOrdered(3, key=lambda x: -x[1]))

[('tu', 0.6020599913279624), ('nombre', 0.6020599913279624), ('claudia', 0.6020599913279624)]


### **TF-IDF** (Inverse Document Frequency): 
    
### $$tfidf(t,d,D) = tf(t,d) \cdot idf(t,D)$$

In [13]:
# Funcion para encontrar el tfidf de una palabra en cada documento
aux = list(IDF_corpus.collect())
def tf_idf(TF):
    """Permite hallar el TF-IDF(Inverse Document Frecuency) de las palabras de un corpus.
    Args:
        TF (list): Lista de listas con los TF's por documento.
    Returns:
        (RDD list): Una lista de listas de tuplas compuesto por (palabra, TF-IDF).
                    Cada lista de la lista, representa un documento
    """
    def find_idf(palabra):
        """Permite hallar el IDF(Inverse Document Frecuency) de las palabras de un corpus.
        Args:
            palabra (string) : Palabra de la cual queremos obtener el IDF.
        Returns:
            valor (float): Valor de idf de la palabra.
        """
        # Para cada palabra, la buscamos en el arreglo idf, para recuperar su valor, usando la funcion filter
        valor = list(filter(lambda x: x[0]==palabra,aux))
        return valor[0][1]
    # Para cada palabra de cada documento almacenado en el TF, obtenemos el TF-IDF
    result = TF.map(lambda x: [(word[0],word[1]*float(find_idf(word[0]))) for word in x])
    return result

In [14]:
TF_IDF_CORPUS = tf_idf(TF)

In [15]:
# Mostramos los valores del TF-IDF, para las palabras de cada documento
for i in (TF_IDF_CORPUS.collect()):
    print(i)

[('hola', 0.2342468675289098), ('como', 0.11979187908506814), ('estas', 0.11979187908506814)]
[('hola', 0.013774377185074694), ('buenos', 0.02754875437014939), ('dias', 0.02754875437014939), ('bien', 0.20878687094906243), ('y', 0.01820873619052574), ('tu', 0.02754875437014939), ('como', 0.01820873619052574), ('estas', 0.01820873619052574)]
[('hola', 0.0291728207956116), ('mi', 0.0583456415912232), ('nombre', 0.0583456415912232), ('es', 0.0583456415912232), ('claudia', 0.0583456415912232), ('y', 0.03856437141683326), ('el', 0.0583456415912232), ('tuyo', 0.0583456415912232)]


### 4. N-Grams
---

In [16]:
def ngrams(n,listaTokens):
    """Permite hallar una lista con gramas de tamaño n.
        Args:
            listaTokens (array) : Lista de palabras preprocesadas, obtenidas de un texto.
        Returns:
            n_grams (array): Lista de gramas de tamaño n para la lista de tokens.
    """
    particiones=zip(*[listaTokens[i:] for i in range(0,n)])
    # Para todas las particiones posibles, juntamos las palabras de cada una, segun el valor de n
    n_grams=[' '.join(ngram) for ngram in particiones]
    return n_grams

In [17]:
# Usaremos el preprocesado del item 1 como corpus
tokens_x_texto = preprocesado.map(lambda x: x.split())
print(tokens_x_texto.collect())

[['hola', 'hola', 'hola', 'como', 'estas'], ['hola', 'buenos', 'dias', 'bien', 'bien', 'y', 'tu', 'como', 'estas'], ['hola', 'mi', 'nombre', 'es', 'claudia', 'y', 'el', 'tuyo']]


In [18]:
# Aplicamos la funcion de n_grams a cada documento del corpus
# Ṕara n grams con n=2
n_grams_docs = tokens_x_texto.map(lambda x: ngrams(2,x))
print(n_grams_docs.collect())


[['hola hola', 'hola hola', 'hola como', 'como estas'], ['hola buenos', 'buenos dias', 'dias bien', 'bien bien', 'bien y', 'y tu', 'tu como', 'como estas'], ['hola mi', 'mi nombre', 'nombre es', 'es claudia', 'claudia y', 'y el', 'el tuyo']]


In [19]:
# Aplicamos la funcion de n_grams a cada documento del corpus
# Para n grams con n=3
n_grams_docs = tokens_x_texto.map(lambda x: ngrams(3,x))
print(n_grams_docs.collect())

[['hola hola hola', 'hola hola como', 'hola como estas'], ['hola buenos dias', 'buenos dias bien', 'dias bien bien', 'bien bien y', 'bien y tu', 'y tu como', 'tu como estas'], ['hola mi nombre', 'mi nombre es', 'nombre es claudia', 'es claudia y', 'claudia y el', 'y el tuyo']]
